### <center> Upload Best Image and Polygon Mask to Roboflow</center>

In [1]:
# import necessary packages
import os
import pickle
import cv2
import numpy as np
import subprocess
import exiftool
import pandas as pd
import PIL
import json
import matplotlib.pyplot as plt

In [2]:
# Check if ExifTool is accessible
try:
    subprocess.run(["exiftool", "-ver"], check=True)
    print("ExifTool is accessible.")
except subprocess.CalledProcessError:
    print("ExifTool is not accessible.")
print()

ExifTool is accessible.



In [3]:
# Working directory for IDP outputs
dir = "C:/Users/exx/EasyIDP/Route9_Orchard4/Outputs/"

# Path to folder containing raw UAV images
raw_img_folder_path = "D:/Savanna Institute Drone 2023/Route 9/Raw Images/Orchard 4/20230823_Route9-Orchard4"

In [4]:
# Read the folder names in "C:\Users\zack\Desktop\easyIDP\Route9_Orchard3\best5_images". Store the folder names in a list
folder_path = dir + "best5_images"
folder_names = os.listdir(folder_path)

# remove non integer names from the list
tree_ID = [int(name) for name in folder_names if name.isdigit()]

In [5]:
 # Tree ID is the key and the image names are the values.
idp_img_names = {}
for i in range(len(tree_ID)):
    idp_img_names[tree_ID[i]] = os.listdir(folder_path + "/" + folder_names[i])

In [6]:
# Paths to folders containing IDP outputs from reverse projection
idp_img_path = {}
for i in range(len(tree_ID)):
    idp_img_path[tree_ID[i]] = [folder_path + "/" + folder_names[i] + "/" + idp_img_names[tree_ID[i]][j] for j in range(len(idp_img_names[tree_ID[i]]))]

In [7]:
# Calculate sharpness and contrast for each idp image. 
def sharpness_contrast(img_path):
    # read image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Sharpness is the Laplacian of the image gradients.
    sharpness = cv2.Laplacian(img, cv2.CV_64F).var()
    # Contrast is the standard deviation of the pixel values in greyscale (RMS contrast).
    contrast = np.std(img)
    return sharpness, contrast

In [8]:
# Modify the image names to match the raw UAV image names. 
uav_img_names = {}

# Remove "{treeID}_" from the start of every image name. 
for key in idp_img_names.keys():
    uav_img_names[key] = [name.split("_", 1)[1] for name in idp_img_names[key]]

# Remove rest of image name after "_D"
for key in uav_img_names.keys():
    uav_img_names[key] = [name.split("_at", 1)[0] for name in uav_img_names[key]]

# Add ".JPG" to the end of each image name
for key in uav_img_names.keys():
    uav_img_names[key] = [name + ".JPG" for name in uav_img_names[key]]

In [9]:
# Store image names as full path. Image directory is raw_img_folder_path
uav_img_path = {}
for key in uav_img_names.keys():
    uav_img_path[key] = [raw_img_folder_path + "/" + name for name in uav_img_names[key]]

In [10]:
# Function to calculate gimbal pitch using exiftool. 
# Gimbal pitch is the angle of the camera from the horizontal plane.
def gimbal_pitch(img_path):
    with exiftool.ExifToolHelper() as et:
        for d in et.get_tags(img_path, tags=["GimbalPitchDegree"]):
            return d

In [11]:
# Calculate sharpness and contrast for all cropped images (the outputs from easyIDP backward projection).
sharpness_contrast_cropped = {}
for key in idp_img_path.keys():
    sharpness_contrast_cropped[key] = [sharpness_contrast(img) for img in idp_img_path[key]]

In [12]:
# Calculate gimbal pitch for all UAV images.
gimbal_pitch_uav = {}
for key in uav_img_path.keys():
        gimbal_pitch_uav[key] = [gimbal_pitch(img) for img in uav_img_path[key]]
        gimbal_pitch_uav[key] = [list(d.values())[1] for d in gimbal_pitch_uav[key]]

In [13]:
# Prioritize gimbal pitch unless sharpness OR contrast highest value is more than 15% larger than 
# gimbal pitch == 90 (rounded) sharpness or contrast. If no gimbal pitch values are equal to 90 (rounded),
# or contrast and sharpness are more than 15% larger than gimbal pitch == 90 (rounded), then the image 
# with the highest sharpness is selected. If sharpness values are within 15% of each other, then the 
# image with the highest contrast is selected.

def best_img(uav_img_path, sharpness_contrast_cropped, gimbal_pitch_uav):
    # store image paths, sharpnes, contrast, and gimbal pitch of every image containing treeID
    img_data = []
    for i in range(len(uav_img_path)):
        img_data.append([uav_img_path[i], sharpness_contrast_cropped[i][0], sharpness_contrast_cropped[i][1], gimbal_pitch_uav[i]])
    # convert img_data to a pandas dataframe
    df = pd.DataFrame(img_data, columns=["img_path", "sharpness", "contrast", "gimbal_pitch"])
    
    # Find the images with gimbal pitch == abs(90) when rounded to the nearest whole number
    gimbal_pitch_90 = df.loc[df["gimbal_pitch"].abs().round() == 90]

    # if multiple images have gimbal pitch == abs(90), or if largest sharpness | contrast for tree 
    # is more than 15% larger than the largest sharpness/contrast value for gimbal pitch == 90,
    # best image is the image with the highest sharpness. 
    if len(gimbal_pitch_90) == 0 or (df["sharpness"].max() - gimbal_pitch_90["sharpness"].max() > 0.15 * df["sharpness"].max()) or (df["contrast"].max() - gimbal_pitch_90["contrast"].max() > 0.15 * df["contrast"].max()):
        best_image = df.loc[df["sharpness"] == df["sharpness"].max(), "img_path"].values[0]

        # is the next highest sharpness value within 15% of the highest sharpness value?
        if df["sharpness"].max() - df["sharpness"].nlargest(2).iloc[-1] <= 0.15 * df["sharpness"].max():
            
            # if so, best image is the image with the highest contrast
            max_contrast = df["contrast"].max()
            best_image = df.loc[df["contrast"] == max_contrast, "img_path"].values[0]

    # if not, best image is the image with the highest gimbal pitch == abs(90)
    else:
        best_image = gimbal_pitch_90.loc[gimbal_pitch_90["gimbal_pitch"] == gimbal_pitch_90["gimbal_pitch"], "img_path"].values[0]

    return best_image

In [14]:
# for each tree ID, find the best UAV image
best_images = {}
for key in uav_img_path.keys():
    best_images[key] = best_img(uav_img_path[key], sharpness_contrast_cropped[key], gimbal_pitch_uav[key])

In [15]:
# Load the pixel coordinates dictionary of best 5 images for each tree canopy from pkl file
with open(dir + "ROI_pixelCoords_best5.pkl", "rb") as f:
    pixelCoords = pickle.load(f)

In [16]:
# add .JPG to end of image keys in pixel coords dictionary for each tree ID
for key in pixelCoords.keys():
    pixelCoords[key] = {k + ".JPG": v for k, v in pixelCoords[key].items()}

In [17]:
# best_images dict contains the file path of the best image for each tree ID. Use the file path to get the image name.
best_image_names = {}
for key in best_images.keys():
    best_image_names[key] = best_images[key].split("/")[-1]

In [18]:
# Use best image name to get corresponding pixel coordinates from pixelCoords dictionary
best_image_pixelCoords = {}
for key in best_image_names.keys():
    best_image_pixelCoords[key] = pixelCoords[f'{key}'][best_image_names[key]]

In [19]:
# function for reshaping polygons 
def get_polygon(polygons):
    polygons = np.array(polygons)
    polygons = polygons.astype('float').reshape(-1, 2)
    if polygons.shape[0] == 1 : return polygons
    return np.squeeze(polygons)

# function for plotting image
def img_show(image, ax = None, figsize = (6, 9)):
    if ax is None:
        fig, ax = plt.subplots(figsize = figsize)
    ax.imshow(image)
    # ax.xaxis.tick_top()
    ax.axis('off')
    return ax

# Function for plotting mask
def plot_mask(ax, polygons):
    ax.plot(polygons[:, 0], polygons[:, 1], c = 'y', linewidth = 0.7, alpha = 0.8)
    return ax

# function for plotting image with mask
def plot_img_mask(image, polygon):
    ax = img_show(image)
    polygon = get_polygon(polygon)
    plot_mask(ax, polygon)
    return ax

# modify get_bbox function to buffer the bounding box
def get_buffered_bbox(image, polygon, buffer):
    xmin = np.min(polygon[:, 0])
    xmax = np.max(polygon[:, 0])
    ymin = np.min(polygon[:, 1])
    ymax = np.max(polygon[:, 1])
    bbox = [max(0, xmin - buffer), max(0, ymin - buffer), min(xmax + buffer, image.size[0]), min(ymax + buffer, image.size[1])]
    return bbox

# Function to crop image using buffered bounding box
def crop_image_bbox(image, bbox, buffer):
    xmin, ymin, xmax, ymax = bbox
    xmin = max(0, xmin - buffer)
    ymin = max(0, ymin - buffer)
    xmax = min(image.size[0], xmax + buffer)
    ymax = min(image.size[1], ymax + buffer)
    return image.crop((xmin, ymin, xmax, ymax))

# modify polygon coords to reflect crop
def crop_polygon(image, polygon, bbox, buffer):
    xmin, ymin, xmax, ymax = bbox
    xmin = max(0, xmin - buffer)
    ymin = max(0, ymin - buffer)
    xmax = min(image.size[0], xmax + buffer)
    ymax = min(image.size[1], ymax + buffer)
    polygon[:, 0] = np.clip(polygon[:, 0], xmin, xmax)
    polygon[:, 1] = np.clip(polygon[:, 1], ymin, ymax)
    polygon[:, 0] -= xmin
    polygon[:, 1] -= ymin
    return polygon

# function to crop image using buffered bounding box
def crop_image(image, polygon, buffer):
    bbox = get_buffered_bbox(image, polygon, buffer)
    cropped_image = crop_image_bbox(image, bbox, buffer)
    cropped_polygon = crop_polygon(image, polygon, bbox, buffer)
    return cropped_image, cropped_polygon


# Calculate bbox in COCO format of cropped polygon
def get_coco_bbox(polygon):
    xmin = np.min(polygon[:, 0])
    xmax = np.max(polygon[:, 0])
    ymin = np.min(polygon[:, 1])
    ymax = np.max(polygon[:, 1])
    bbox = [xmin, ymin, xmax - xmin, ymax - ymin]
    return bbox

In [20]:
# create folder in dir for storing raw UAV images in .PNG format. 
png_folder = dir + "best_image_PNGs/"
if not os.path.exists(png_folder):
    os.makedirs(png_folder)

# create folder in dir for storing cropped images in .PNG format.
cropped_png_folder = dir + "Roboflow/images/"
if not os.path.exists(cropped_png_folder):
    os.makedirs(cropped_png_folder)

# create folder in dir for storing annotation JSON
annotation_folder = dir + "Roboflow/annotations/"
if not os.path.exists(annotation_folder):
    os.makedirs(annotation_folder)

In [21]:
# for each tree ID, crop the best image and save the cropped image with cropped polygon mask
cropped_polygons = {}
for key in best_image_pixelCoords.keys():
    image = PIL.Image.open(best_images[key])
    # copy the best image as png with transparent background and same dpi as original image in temp folder
    image.save(png_folder + str(key) + ".png", format = "PNG", dpi = (image.info["dpi"][0], image.info["dpi"][1]))
    # load best image png
    image = PIL.Image.open(png_folder + str(key) + ".png")
    polygon = best_image_pixelCoords[key]
    buffer = int(200)
    cropped_image, cropped_polygon = crop_image(image, polygon, buffer)
    cropped_polygons[key] = cropped_polygon
    # save cropped image as png with transparent background and same dpi as original image in cropped_png_folder
    cropped_image.save(cropped_png_folder + str(key) + ".png", format = "PNG", dpi = (image.info["dpi"][0], image.info["dpi"][1]))

In [22]:
# Save the best image and corresponding cropped image polygon coordinates to a JSON file in COCO format. 
# COCO JSON file includes the following fields: 

# “info”: This field contains metadata about the dataset, such as the version, description, and contributor information

# “licenses”: This field contains information about the licenses associated with the images and videos in the dataset

# “images”: This field contains a list of dictionaries, each representing an image in the dataset. Each dictionary includes the following fields:
#     “id”: The unique identifier for the image (i.e., key/treeID)
#     “width”: The width of the image in pixels
#     “height”: The height of the image in pixels
#     “file_name”: The file name of the image
#     “license”: The license associated with the image
#     “date_captured”: The date the image was captured (optional)

# “annotations”: This field contains a list of dictionaries, each representing an annotation for an image in the dataset. Each dictionary includes the following fields:
#     “id”: The unique identifier for the annotation (i.e., key/treeID)
#     “image_id”: The identifier for the image to which the annotation belongs
#     “category_id”: The identifier for the category (i.e., class) to which the annotation belongs
#     “bbox”: The bounding box for the annotation (i.e., bounding box of the canopy polygon in x,y,w,h format)
#     “area”: The area of the bbox in square pixels (w * h)
#     “segmentation”: The segmentation mask for the annotation (i.e., canopy polygon)
#     “iscrowd”: A binary flag indicating whether the annotation represents a single object or a group of objects)
#     “attributes”: Additional attributes associated with the annotation (optional)

###-------------------------------------------------------------------------------------------------------------------------------------------------------------------------###

# create the COCO JSON file
coco_data = {}
coco_data["info"] = {
    "description": "Route 9 Orchard 4",
    "version": "1.0",
    "contributor": "Zack Loken",
    "date_created": "2024-07-04"
}

coco_data["licenses"] = [
    {
        "url": "https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en",
        "id": 1,
        "name": "Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License"
    }
]

coco_data["images"] = []
coco_data["annotations"] = []

# add cropped best_image data from cropped_png_folder + str(key) + ".png") to coco_data["images"]
for key in best_images.keys():
    image = PIL.Image.open(cropped_png_folder + str(key) + ".png")
    image_data = {
        "id": key,
        "width": image.size[0],
        "height": image.size[1],
        "file_name": str(key) + ".png",
        "license": 1,
        "date_captured": "08-23-2023"
    }
    coco_data["images"].append(image_data)

# add cropped_polygon data from cropped_polygons[key] to coco_data["annotations"]
for key in cropped_polygons.keys():
    cropped_polygon = cropped_polygons[key]
    # Calculate bbox in COCO format of cropped polygon
    bbox = get_coco_bbox(cropped_polygon)
    area = bbox[2] * bbox[3]
    annotation_data = {
        # id is original uav image name
        "id": uav_img_names[key][0],
        "image_id": key,
        "category_id": "Canopy",
        "bbox": bbox,
        "area": area,
        "segmentation": [cropped_polygon.flatten().tolist()],
        "iscrowd": False,
        "attributes": {}
    }
    coco_data["annotations"].append(annotation_data)

# save coco_data to a JSON file
with open(annotation_folder + "canopyMasks.coco.json", "w") as f:
    json.dump(coco_data, f)

##### Upload best cropped images and corresponding canopy polygons to Roboflow

In [23]:
# Upload the images and annotations to Roboflow
import glob
from roboflow import Roboflow

# API Key for project workspace:
api_key = "5dM5PdVffGR3ONX8CeRu"

# Initialize Roboflow client
rf = Roboflow(api_key=api_key)

# Retrieve your current workspace and project name
print(rf.workspace())

# annotation file path
annotation_path = annotation_folder + "canopyMasks.coco.json"

project = rf.workspace('chestnut-detection').project('route-9-orchard-4')

# Upload images and annotations to Roboflow
image_glob = glob.glob(cropped_png_folder + '/*' + ".png")
for image_path in image_glob:
    print(project.single_upload(image_path = image_path, 
                         annotation_path = annotation_path,
                         num_retry_uploads = 3))

loading Roboflow workspace...
{
  "name": "Chestnut Detection",
  "url": "chestnut-detection",
  "projects": [
    "chestnut-detection/route-9-orchard-3",
    "chestnut-detection/route-9-orchard-4"
  ]
}
loading Roboflow workspace...
loading Roboflow project...
{'image': {'duplicate': True, 'id': 'Z9stCW7rAMFMhWlAtUF5'}, 'annotation': {'error': UploadError("save annotation for Z9stCW7rAMFMhWlAtUF5 / bad response: 400: {'message': 'Image was already annotated.', 'type': 'InvalidImageException', 'hint': 'This image was already annotated; to overwrite the annotation, pass overwrite=true as a query parameter.'}")}, 'upload_time': 3.3167965412139893, 'annotation_time': 12.567750453948975, 'upload_retry_attempts': 0}
{'image': {'duplicate': True, 'id': 'HTrum1Om01tpW97xxDUs'}, 'annotation': {'error': ConnectionError(ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))}, 'upload_time': 3.34181332